In [ ]:
# import stuff and define functions

In [ ]:
import pandas as pd
import subprocess
import os
import shutil
import re

In [ ]:
def get_the_table_data():
    for f in os.listdir():
        if f.endswith(".csv"):
                video_schnitt_df = pd.read_csv(f)
                print(f"Table {f} loaded succesfully")
                return video_schnitt_df
    raise ValueError("There is no csv in this directory")

In [ ]:
def get_video_duration(file_path):
    command = ['ffmpeg', '-i', file_path]
    result = subprocess.run(command, text=True, capture_output=True)
    output_lines = result.stderr.split('\n')
    duration_line = [line for line in output_lines if 'Duration' in line][0]
    duration = duration_line.strip().split(",")[0].split(" ")[1]
    return duration

In [ ]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    df = df.astype(str)
    print("Table Columns cleaned")
    return df

In [ ]:
def cut_head_tail(original_video,output_file,cut_head="00:00:00",cut_tail="00:50:00"):

    # Construct the command
    command = [
        'ffmpeg',
        '-i', original_video,
        '-ss', cut_head,
        '-to', cut_tail,
        '-c:v','libx264',
        '-c:a', 'copy',
        output_file
    ]
    
     # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [ ]:
def create_folder(folder_name):
    try:
        # Create a new folder in the current working directory
        os.mkdir(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    except FileExistsError:
        print(f"Folder '{folder_name}' already exists.")

In [ ]:

def cut_in_2_pieces(input_file, timestamp, head_output, tail_output):
    
    # -ss = ab dort; -to = bis dort
    head_cmd = ['ffmpeg', '-i', input_file, '-to', timestamp, '-c:v', 'copy', '-c:a', 'copy', head_output]
    tail_cmd = ['ffmpeg', '-i', input_file, '-ss', timestamp, '-c:v', 'copy', '-c:a', 'copy', tail_output]

# Execute the commands
    try:
        subprocess.run(head_cmd, check=True)
        subprocess.run(tail_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [ ]:
def get_columns_to_divide(df):
    pattern = re.compile(r'^teilen\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [ ]:
def calculate_time(*timestamps,kind="addition"):
    """
    Calculate time based on the given timestamps and operation type.

    Parameters:
    - timestamps (str): Variable number of timestamps.
    - kind (str): The operation type. Allowed values are 'addition' and 'subtraction'.
    """
    
    #addition
    if kind == "addition":
        stamp1 = None
        for timestamp in timestamps:
        
        # calculate timestamp
            if stamp1 is None:
                stamp1 = timestamp

            else:
                timedelta = stamp1 + pd.to_timedelta(timestamp)
                dummy_date = pd.Timestamp('1900-01-01')
                new_timestamp = dummy_date + timedelta
                timestamp = new_timestamp.strftime('%H:%M:%S.%f')[:-3]
                return timestamp
    
    #subtraction
    elif kind == "subtraction":
        stamp1 = None
        for timestamp in timestamps:
        
        # calculate timestamp
            if stamp1 is None:
                stamp1 = timestamp

            else:
                timedelta = stamp1 - pd.to_timedelta(timestamp)
                dummy_date = pd.Timestamp('1900-01-01')
                new_timestamp = dummy_date + timedelta
                timestamp = new_timestamp.strftime('%H:%M:%S.%f')[:-3]
                return timestamp        

In [ ]:
# get file and clean

In [ ]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)

In [ ]:
# cut the video first if needed

In [ ]:
# # videos need to be in same directory with python script
# try:
#     for idx, video_name in video_schnitt_df["dateiname"].items():

#         if video_schnitt_df["schnitt_setzen_bei"][idx] == "nan":
#             continue

#         # um diesen Loop an neue CSV anzupassen kann direkt ein infinite loop a la cut_things_out her

#         # define arguments
#         output_file = f"{video_name.split('.')[0]}_ohne_start_ende_cut_and_keep.mp4"
#         cut_head = video_schnitt_df["vorne_abschneiden_bis"][idx]
#         cut_tail = video_schnitt_df["hinten_abschneiden_ab"][idx]

#         #building logic when just to cut head or tail
#         if cut_head == "nan":
#             cut_head = "00:00:00"
#         if cut_tail == "nan":
#             cut_tail = get_video_duration(video_name)
    
#         print("input name:",video_name,"Schnittpunkte:", cut_head, cut_tail)
#         cut_head_tail(video_name, output_file, cut_head, cut_tail)
# except Exception as error:
#     print("An error occurred:", error)

In [ ]:
# cut and keep the pieces

In [ ]:
#loop through videonames and then through columns to divide
for idx, video_name in video_schnitt_df["dateiname"].items():
    columns_to_divide = get_columns_to_divide(video_schnitt_df)
    for i,col in enumerate(video_schnitt_df[columns_to_divide]):
        
        #rule when there is more then one division
        if video_schnitt_df[col][idx] != "nan":
            if i > 0:
                input_file = f"{video_name.split('.')[0]}_tail{i}.mp4"
                timestamp1 = timestamp
                timestamp2 = video_schnitt_df[col][idx]
                timestamp = calculate_time(timestamp2,timestamp1,kind="subtraction")
                head_output = f"{video_name.split('.')[0]}_müll{i+1}.mp4"
                tail_output = f"{video_name.split('.')[0]}_tail{i+1}.mp4"
                
                print("all Variables second cut:",input_file,timestamp,head_output,tail_output)

                cut_in_2_pieces(input_file,timestamp,head_output,tail_output)
                os.remove(input_file)
            else:

                #define all variables needed
                input_file = video_name
                timestamp = video_schnitt_df[col][idx]
                head_output = f"{video_name.split('.')[0]}_head{i+1}.mp4"
                tail_output = f"{video_name.split('.')[0]}_tail{i+1}.mp4"
                
                print("all variables:",input_file,timestamp,head_output,tail_output)

                #start seperation
                cut_in_2_pieces(input_file,timestamp,head_output,tail_output)

In [ ]:
# create folders and move files

In [ ]:
path_folder_endprodukte = "endprodukte"
create_folder(path_folder_endprodukte)

path_folder_script_output = "script_output"
create_folder(path_folder_script_output)

In [ ]:
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [ ]:
for filename in os.listdir():
    if filename.endswith("tail.mp4"):
        shutil.move(filename,path_folder_endprodukte)
    elif filename.endswith("mp4") and filename not in original_video_names:
        shutil.move(filename,path_folder_script_output)

In [ ]:
for video_name in video_schnitt_df["dateiname"]:
    pattern = re.compile(rf'^{video_name}_head\d+.mp4$')
    for filename in os.listdir():
        if pattern.match(filename):
            print(filename)